## Pré requisitos

In [1]:
!pip install transformers

In [2]:
!pip install torch

In [3]:
!pip install accelerate==0.21.0

In [4]:
!pip install pandas

In [5]:
!pip install scikit-learn

## Leitura da base de dados

In [6]:
import pandas as pd

# Leitura da base de dados
# Parametros adicionais sao necessarios ja que o caractere de separacao da base consisti no ¨
df_perguntas = pd.read_csv("../Dados/dados.csv")
df_perguntas.head()

,texto,materia
0,"olá, como são formadas as cavernas? e as estal...",Química
1,Qual a diferença entre a gordura animal e a go...,Biologia
2,Como funciona a relação do ângulo externo do t...,Matemática
3,(UESB BA/2017) Sejam dois conjuntos não vazios...,Matemática
4,Boa tarde! Qual a relação das mitocôndrias com...,Biologia


In [7]:
# Retirando as classes "target" dinamicamente, no caso as materias das perguntas
materias = df_perguntas['materia'].unique().tolist()

print(materias)
print(f"Total de matérias: {len(materias)}")

['Química', 'Biologia', 'Matemática', 'História', 'Literatura', 'Física']
Total de matérias: 6


In [8]:
for i in range (len(materias)):
    print(f"{materias[i]} tem {len(df_perguntas[df_perguntas.materia == materias[i]])} Questões")

Química tem 11 Questões
Biologia tem 10 Questões
Matemática tem 10 Questões
História tem 10 Questões
Literatura tem 10 Questões
Física tem 10 Questões


### Dicionarios: ID -> Materia e Materia -> ID

In [9]:
NUMERO_MATERIAS = len(materias)

dic_id_para_materia = {id:materia for id,materia in enumerate(materias)} 
dic_materia_para_id = {materia:id for id,materia in enumerate(materias)}

In [10]:
dic_id_para_materia

{0: 'Química',
 1: 'Biologia',
 2: 'Matemática',
 3: 'História',
 4: 'Literatura',
 5: 'Física'}

In [11]:
dic_materia_para_id

{'Química': 0,
 'Biologia': 1,
 'Matemática': 2,
 'História': 3,
 'Literatura': 4,
 'Física': 5}

###

In [12]:
# Adicionando uma coluna extra para que seja mais fácil a contagem de perguntas de X materia
df_perguntas['label'] = pd.factorize(df_perguntas.materia)[0]
df_perguntas.head()

,texto,materia,label
0,"olá, como são formadas as cavernas? e as estal...",Química,0
1,Qual a diferença entre a gordura animal e a go...,Biologia,1
2,Como funciona a relação do ângulo externo do t...,Matemática,2
3,(UESB BA/2017) Sejam dois conjuntos não vazios...,Matemática,2
4,Boa tarde! Qual a relação das mitocôndrias com...,Biologia,1


## Baixando o modelo BERTimbau

In [13]:
from transformers import BertTokenizerFast  # Classe necessaria para o tokenizador, que usaremos para tratamento do texto
from transformers import BertForSequenceClassification  # Classe necessaria para importar o modelo
from transformers import pipeline

# Baixar o modelo pode demorar um pouco
bertimbau = BertForSequenceClassification.from_pretrained('neuralmind/bert-large-portuguese-cased', num_labels=NUMERO_MATERIAS, id2label=dic_id_para_materia, label2id=dic_materia_para_id)
tokenizer = BertTokenizerFast.from_pretrained('neuralmind/bert-large-portuguese-cased', max_lenght=512)

c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-large-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Hardware a ser utilizado para o treino

In [14]:
from torch import cuda

hardware = None

# Verifica se e possivel utilizar a placa de video para o treinamento do PyTorch
if cuda.is_available(): hardware = 'cuda'
else: hardware = 'cpu'

# Printa o hardware que sera utilizado de uma forma mais clara ao usuario
print('Placa de video' if hardware == 'cuda' else 'Processador')

Processador


### Informacoes sobre o modelo BERTimbau

In [15]:
# Informacoes sobre o modelo BERTimbau
bertimbau.to(hardware)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29794, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,

## Treinando nosso modelo

### Primeiro vamos subdividir e preparar os dados

In [16]:
import numpy as np
from sklearn.model_selection import train_test_split

X = df_perguntas.drop(columns=['materia', 'label'])
y = df_perguntas.drop(columns=['materia', 'texto'])

X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.30, shuffle=True)

X_treino = list(X_treino['texto'])
X_teste = list(X_teste['texto'])

y_treino = list(y_treino['label'])
y_teste = list(y_teste['label'])

In [17]:
for label,(chave, materia) in enumerate(dic_id_para_materia.items()):
    print(f"Treino tem {y_treino.count(chave)} questões de {materia}")


Treino tem 6 questões de Química
Treino tem 7 questões de Biologia
Treino tem 9 questões de Matemática
Treino tem 6 questões de História
Treino tem 7 questões de Literatura
Treino tem 7 questões de Física


In [18]:
for label,(chave, materia) in enumerate(dic_id_para_materia.items()):
    print(f"Treino tem {y_teste.count(chave)} questões de {materia}")

Treino tem 5 questões de Química
Treino tem 3 questões de Biologia
Treino tem 1 questões de Matemática
Treino tem 4 questões de História
Treino tem 3 questões de Literatura
Treino tem 3 questões de Física


In [19]:
tokens_treino = tokenizer(X_treino, truncation=True, padding=True, max_length=512)
tokens_teste = tokenizer(X_teste, truncation=True, padding=True, max_length=512)

In [20]:
import torch
from torch.utils.data import Dataset

class PacoteDados(Dataset):
    def __init__(self, tokens, labels):
        self.tokens = tokens
        self.labels = labels
    
    def __getitem__(self, indice):
        item = {key: torch.tensor(val[indice]) for key, val in self.tokens.items()}
        item['label'] = torch.tensor(self.labels[indice])
        return item
    
    def __len__(self):
        return len(self.labels)


In [21]:
pacote_treino = PacoteDados(tokens_treino, y_treino)
pacote_teste = PacoteDados(tokens_teste, y_teste)

In [22]:
from transformers import TrainingArguments, Trainer

argumentos_treino = TrainingArguments(
    #* Diretorio de saida serve para guardar previsoes e checkpoints do modelo
    output_dir = "../Model/Dados", 
    overwrite_output_dir = True,
    do_train = True,
    learning_rate=5e-5,

    #* do_eval nos diz se o modelo deve ou nao fazer a etapa de validacao
    do_eval = True,
    
    #* Numero de epocas, normalmente 3
    num_train_epochs = 8, # Melhor resultado variando entre 8 e 10
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 16,
    # Number of steps used for a linear warmup
    #warmup_steps = 100,                
    #weight_decay = 0.01,
    logging_strategy = 'steps',

    #* Diretorio para armazenamento de Logs                 
    logging_dir = '../Model/Logs/',            
    #logging_steps = 50,
    evaluation_strategy = "steps",
    #eval_steps = 50,
    save_strategy = "steps", 
    fp16 = True if hardware == 'cuda' else False,
    load_best_model_at_end = True
)

In [23]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calcular_metricas(previsoes):
    labels = previsoes.label_ids
    
    previsao = previsoes.predictions.argmax(-1)
    
    precisao, recall, f1, _ = precision_recall_fscore_support(labels, previsao, average='macro')
    acuracia = accuracy_score(labels, previsao)
    
    return {
        'Acuracia': acuracia,
        'F1': f1,
        'Precisao': precisao,
        'Recall': recall
    }

def predizer(texto):
    # Faz o texto ser tokenzinado e transformado em tensores para o PyTorch
    inputs = tokenizer(texto, padding=True, truncation=True, max_length=512, return_tensors='pt').to(hardware)

    # Passa os tensores como input para o nosso modelo
    resultado = bertimbau(inputs)

    # Primeiro pegamos da tupla resultado somente o tensor que queremos na posicao 0
    # Entao aplicamos o softmax para que possamos ter as probabilidades de cada classe
    probabilidades = resultado[0].softmax(1)

    # Assim conseguimos pegar o maior valor das probabilidades, ou seja a classe mais provavel
    id_label_prevista = probabilidades.argmax()

    # Entao podemos pegar a label prevista atraves das configuracoes que colocamos ao definir o modelo
    # Assim usamos o ID obtido para conseguir o nome da materia prevista
    label_prevista = bertimbau.config.id2label[id_label_prevista.item()]

    return probabilidades, id_label_prevista, label_prevista


In [24]:
treinador = Trainer(
    model = bertimbau,
    args = argumentos_treino,
    train_dataset = pacote_treino,
    eval_dataset= pacote_teste,
    compute_metrics = calcular_metricas
)

In [25]:
treinador.train()

  0%|          | 0/48 [00:00<?, ?it/s]

100%|██████████| 48/48 [11:48<00:00, 14.76s/it]

{'train_runtime': 708.2474, 'train_samples_per_second': 0.474, 'train_steps_per_second': 0.068, 'train_loss': 0.7534275849660238, 'epoch': 8.0}


TrainOutput(global_step=48, training_loss=0.7534275849660238, metrics={'train_runtime': 708.2474, 'train_samples_per_second': 0.474, 'train_steps_per_second': 0.068, 'train_loss': 0.7534275849660238, 'epoch': 8.0})

In [26]:
treinador.predict(pacote_teste).metrics

100%|██████████| 2/2 [00:01<00:00,  1.36it/s]


{'test_loss': 0.5556139349937439,
 'test_Acuracia': 0.8421052631578947,
 'test_F1': 0.8277777777777776,
 'test_Precisao': 0.85,
 'test_Recall': 0.8777777777777778,
 'test_runtime': 8.282,
 'test_samples_per_second': 2.294,
 'test_steps_per_second': 0.241}

## Salvando o modelo

In [51]:
path_modelo = "../Model/Modelo"
#! Somente salvar se o modelo for melhor que o anterior
treinador.save_model(path_modelo)
tokenizer.save_pretrained(path_modelo)

('../Model/Modelo\\tokenizer_config.json',
 '../Model/Modelo\\special_tokens_map.json',
 '../Model/Modelo\\vocab.txt',
 '../Model/Modelo\\added_tokens.json',
 '../Model/Modelo\\tokenizer.json')

## Carregando o modelo

In [28]:
path_modelo = "../Model/Modelo"

modelo = BertForSequenceClassification.from_pretrained(path_modelo)
tokenizer = BertTokenizerFast.from_pretrained(path_modelo)
classificador = pipeline("text-classification", model=modelo, tokenizer=tokenizer)

In [35]:
print(classificador("Império romano"))
print(classificador("Me ajude a fazer essa conta: x2 + y2 = 4 + 4"))
print(classificador("Modelo atômico"))
print(classificador("Quando ocorreu o Trovadorismo?"))
print(classificador("Qual a velocidade média de uma pessoa que anda a 2km/h?"))

[{'label': 'História', 'score': 0.46252572536468506}]
[{'label': 'Matemática', 'score': 0.9501256346702576}]
[{'label': 'Química', 'score': 0.802685558795929}]
[{'label': 'Literatura', 'score': 0.9064043164253235}]
[{'label': 'Física', 'score': 0.8219466209411621}]


## Bônus do desafio

In [43]:
import pandas as pd
espacamento = 60

df_perguntas = pd.read_csv("../Dados/dados.csv")
materias = df_perguntas['materia'].unique().tolist()
del df_perguntas

def pegarMateria():
    materia = None
    
    while materia == None:
        for i in range(len(materias)):
            print(f"{i} - {materias[i]}".rjust(espacamento), flush=True)
        
        materia = int(input())

        if (materia >= len(materias) and materia < 0):
            materia = None
            print("Nenhuma matéria encontrada com esse numero")
    
    return materia

In [50]:
import time

quebrarWhile = False
opcao = None

while quebrarWhile == False:
    print("* Menu *".center(espacamento, '-'))
    print("1 - Fazer pergunta ".ljust(espacamento, '-'))
    print("0 - Sair ".ljust(espacamento, '-'), flush=True)
    opcao = int(input())

    if opcao == 0: quebrarWhile = True
    elif opcao == 1:
        print(" Digite sua pergunta ".center(espacamento, '-'), flush=True)
        texto = input()

        print("".center(espacamento, '-'))
        print(" Agora nos diga o numero da matéria ".center(espacamento, '-'), flush=True)
        indice_materia = pegarMateria()

        tempo_inicio = time.perf_counter()
        resultado = classificador(texto)[0]
        tempo_fim = time.perf_counter()

        print("".center(espacamento, "-"))
        print(f"Matéria original: {materias[indice_materia]}")
        print(f"Matéria prevista: {resultado['label']}")
        print(f"Tempo de resposta: {tempo_fim - tempo_inicio} segundos")

        if resultado['label'] == materias[indice_materia]:
            print("A IA acertou a matéria!")
        else:
            print("A IA errou a matéria, talvez precise de um melhor treinamento")
        

    else: print(f"Opção {opcao} não existe no Menu")
    print("\n")

print("Fim da execução!")

--------------------------* Menu *--------------------------
1 - Fazer pergunta -----------------------------------------
0 - Sair ---------------------------------------------------
------------------- Digite sua pergunta --------------------
------------------------------------------------------------
------------ Agora nos diga o numero da matéria ------------
                                                 0 - Química
                                                1 - Biologia
                                              2 - Matemática
                                                3 - História
                                              4 - Literatura
                                                  5 - Física
------------------------------------------------------------
Matéria original: História
Matéria prevista: História
Tempo de resposta: 0.14891679999709595
A IA acertou a matéria!


--------------------------* Menu *--------------------------
1 - Fazer pergunta --------